# BERT Review Polarity

## Setup

In [ ]:
!pip install transformers

In [66]:
from google.colab import drive
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [94]:
PATH_GDRIVE = '/content/drive'
drive.mount(PATH_GDRIVE, force_remount=True)

Mounted at /content/drive


In [93]:
# paths
DIR = 'drive/MyDrive/MSc Dissertation/data/'
PATH_SAMPLES = DIR + 'samples/review_polarity/%s.csv'
PATH_MODELS = DIR + 'models/review_polarity/%s_%d'
PATH_RESULTS = DIR + 'results/review_polarity/%s_%d'
# bert models
BERT_MODEL_ENG = 'distilbert-base-uncased'
BERT_MODEL_NENG = 'distilbert-base-multilingual-cased'

## Data Reading and Writing

In [36]:
def read_data(sample_name, train_size=0.8, test_size=0.5, seed=None):
    df = pd.read_csv(PATH_SAMPLES % sample_name)[['polarity', 'text']].reset_index(drop=True)
    train, remaining = train_test_split(df, train_size=train_size, random_state=seed)
    validation, test = train_test_split(remaining, test_size=test_size, random_state=seed)
    return train, validation, test

In [90]:
def write_data_train(sample_name, run_num, train_history):
    with open((PATH_RESULTS % (sample_name, run_num)) + '_train.txt', 'w+') as f:
        f.write(str(train_history.params) + '\n')
        f.write(str(train_history.history) + '\n')

In [89]:
def write_data_test(sample_name, run_num, Y_test, Y_pred):
    with open((PATH_RESULTS % (sample_name, run_num)) + '_test.txt', 'w+') as f:
        f.write(str(Y_test) + '\n')
        f.write(str(Y_pred) + '\n')

## Model Training and Testing


In [91]:
def train_model(sample_name, train_data, validation_data, run_num=0, num_epochs=2, batch_size=16, learning_rate=5e-5, seed=None):
    with tpu_strategy.scope():
        # load pre-trained stuff
        if sample_name.startswith('eng'): model_name = BERT_MODEL_ENG
        else: model_name = BERT_MODEL_NENG
        tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
        classifier = TFDistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)
        # separate data
        Xt, Yt = train_data['text'].tolist(), train_data['polarity'].tolist()
        Xv, Yv = validation_data['text'].tolist(), validation_data['polarity'].tolist()
        # encode and format data
        encode = lambda x: tokenizer(x, truncation=True, padding=True, return_tensors='tf')
        format = lambda x, y: tf.data.Dataset.from_tensor_slices((dict(x), y)).shuffle(100, seed=seed)
        train_data = format(encode(Xt), Yt)
        validation_data = format(encode(Xv), Yv)
        # train and save the model
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-8)
        loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        classifier.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    fit_results = classifier.fit(
        train_data.batch(batch_size),
        epochs=num_epochs,
        batch_size=batch_size,
        validation_data=validation_data.batch(batch_size)
    )
    classifier.save_pretrained(PATH_MODELS % (sample_name, run_num))
    write_data_train(sample_name, run_num, fit_results)

In [92]:
def test_model(sample_name, test_data, run_num=0, batch_size=16, seed=None):
    with tpu_strategy.scope():
        # load pre-trained stuff
        if sample_name.startswith('eng'): model_name = BERT_MODEL_ENG
        else: model_name = BERT_MODEL_NENG
        tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
        classifier = TFDistilBertForSequenceClassification.from_pretrained(PATH_MODELS % (sample_name, run_num), num_labels=2)
        # separate data
        Xt, Yt = test_data['text'].tolist(), test_data['polarity'].tolist()
        # encode, format and convert data
        encode = lambda x: tokenizer(x, truncation=True, padding=True, return_tensors='tf')
        format = lambda x: tf.data.Dataset.from_tensor_slices(dict(x))
        convert = lambda y: tf.argmax(tf.nn.softmax(y.logits, axis=1), axis=1).numpy()[::8]
        Xt = format(encode(Xt))
        Yp = convert(classifier.predict(Xt)).tolist()
    write_data_test(sample_name, run_num, Yt, Yp)

## Running Everything

In [101]:
batch_sizes = [16, 32, 64]
num_epochs = [2, 3, 4]
run_num = 0
sample_name = 'eng_eq_any_100000'
seed = 1337
train, validation, test = read_data(sample_name, seed=seed)

In [ ]:
for batch_size in batch_sizes:
    for num_epoch in num_epochs:
        print(f'=== RUN_NUM={run_num} ===')
        train_model(sample_name, train, validation, run_num=run_num, seed=seed, batch_size=batch_size, num_epochs=num_epoch)
        test_model(sample_name, test, run_num=run_num, seed=seed, batch_size=batch_size)
        run_num += 1

In [103]:
sample_names = [
    'eng_any_any_100000', 'eng_any_long_100000', 'eng_any_short_100000',
    'eng_eq_any_100000', 'eng_eq_long_100000', 'eng_eq_short_100000',
    'any_any_any_100000', 'any_any_long_100000', 'any_any_short_100000',
    'any_eq_any_100000', 'any_eq_long_100000', 'any_eq_short_100000',
]
seed=1337

In [104]:
for sample_name in sample_names:
    train, validation, test = read_data(sample_name, seed=seed)
    train_model(sample_name, train, validation, run_num=10, batch_size=32, seed=seed)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_719']
You should probably TRAIN this model on a down-stream task to be able to use 

Epoch 1/2
2500/2500 [==============================] - 302s 99ms/step - loss: 0.2483 - accuracy: 0.9022 - val_loss: 0.2261 - val_accuracy: 0.9135
Epoch 2/2
2500/2500 [==============================] - 233s 93ms/step - loss: 0.1823 - accuracy: 0.9324 - val_loss: 0.2507 - val_accuracy: 0.9120


Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_739']
You should probably TRAIN this model on a down-stream task to be able to use 

Epoch 1/2
2500/2500 [==============================] - 303s 99ms/step - loss: 0.2481 - accuracy: 0.9017 - val_loss: 0.2219 - val_accuracy: 0.9147
Epoch 2/2
2500/2500 [==============================] - 233s 93ms/step - loss: 0.1729 - accuracy: 0.9383 - val_loss: 0.2525 - val_accuracy: 0.9119


Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_759']
You should probably TRAIN this model on a down-stream task to be able to use 

...

Epoch 1/2
2500/2500 [==============================] - 340s 114ms/step - loss: 0.4727 - accuracy: 0.7760 - val_loss: 0.4204 - val_accuracy: 0.8157
Epoch 2/2
2500/2500 [==============================] - 274s 109ms/step - loss: 0.3478 - accuracy: 0.8573 - val_loss: 0.4483 - val_accuracy: 0.8090


Some layers from the model checkpoint at distilbert-base-multilingual-cased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_919']
You should probably TRAIN this model on a down-stream t

Epoch 1/2
2500/2500 [==============================] - 342s 114ms/step - loss: 0.4169 - accuracy: 0.8119 - val_loss: 0.3444 - val_accuracy: 0.8579
Epoch 2/2
2500/2500 [==============================] - 273s 109ms/step - loss: 0.2860 - accuracy: 0.8884 - val_loss: 0.3614 - val_accuracy: 0.8583


Some layers from the model checkpoint at distilbert-base-multilingual-cased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_939']
You should probably TRAIN this model on a down-stream t

Epoch 1/2
2500/2500 [==============================] - 344s 116ms/step - loss: 0.4856 - accuracy: 0.7699 - val_loss: 0.4489 - val_accuracy: 0.7960
Epoch 2/2
2500/2500 [==============================] - 273s 109ms/step - loss: 0.3676 - accuracy: 0.8466 - val_loss: 0.4817 - val_accuracy: 0.8002


In [105]:
for sample_name in sample_names:
    _, _, test = read_data(sample_name, seed=seed)
    test_model(sample_name, test, run_num=10, batch_size=32, seed=seed)

Some layers from the model checkpoint at drive/MyDrive/MSc Dissertation/data/models/review_polarity/eng_any_any_100000_10 were not used when initializing TFDistilBertForSequenceClassification: ['dropout_719']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at drive/MyDrive/MSc Dissertation/data/models/review_polarity/eng_any_any_100000_10 and are newly initialized: ['dropout_959']
You should probably TRAIN this model on a down-stream t